In [356]:
import xgboost
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
import json
import numpy as np
from PIL import Image

import multiprocessing
from multiprocessing import sharedctypes, Pool

# from facets import dive
from google.cloud import bigquery as bq

import warnings
warnings.filterwarnings('ignore')

In [368]:
num_examples = df_features['EventId'].unique().shape[0]
print(num_examples)

500


In [369]:
img_dim_inches = 3
img_dpi = 90

In [424]:
def create_image(i):
    import matplotlib.pyplot as plt
    import matplotlib.dates
    import matplotlib.dates as mdates
    from matplotlib.backends.backend_agg import FigureCanvasAgg, FigureCanvas
    from matplotlib.figure import Figure
    from matplotlib.pyplot import FormatStrFormatter
    from pandas.plotting import register_matplotlib_converters
    from matplotlib import rcParams
    rcParams.update({'figure.autolayout': True})
    
    _ = register_matplotlib_converters()
    
    fig = plt.Figure(figsize=(img_dim_inches, img_dim_inches), dpi=img_dpi)
    _ = fig.patch.set_facecolor('white')
    _ = fig.patch.set_alpha(1)
    ax = fig.add_subplot(111)
  
    data = df_voltages[df_voltages['EventId'] == df_features.iloc[i]['EventId']]
    min_unix = data['UnixMillis'].min()
    _ = ax.plot(data['UnixMillis'] - min_unix, data['Voltage'], 'r', zorder=3)
    _ = ax.scatter(data['UnixMillis'] - min_unix, data['Voltage'], c='k', s=20, zorder=4)
    _ = ax.set_xlabel('Milliseconds')
    _ = ax.set_ylabel('Voltage (V)')
    _ = ax.grid(linestyle='-.')
    
    canvas = FigureCanvas(fig)
    
    canvas.print_figure('./IMAGES/' + str(i).zfill(5) + '.png', transparent=False, dpi=img_dpi)
    plt.close(fig)
    
    return None

In [425]:
pool = multiprocessing.Pool()
log_norms = pool.map(create_image, range(num_examples))
_ = pool.close()

In [426]:
def is_perfect_square(n):
    x = n // 2
    y = set([x])
    while x * x != n:
        x = (x + (n // x)) // 2
        if x in y: return False
        y.add(x)
    return True

In [427]:
if is_perfect_square(num_examples):
    max_cols = int(np.sqrt(num_examples))
    max_rows = int(np.sqrt(num_examples))
else:
    max_cols = int(np.ceil(np.sqrt(num_examples)))
    max_rows = int(np.ceil(float(num_examples)/max_cols))
    
    
xdim = max_rows*img_dim_inches*img_dpi
ydim = max_cols*img_dim_inches*img_dpi
img_arr = np.zeros([xdim, ydim, 4])

In [428]:
xdim = max_rows*img_dim_inches*img_dpi
ydim = max_cols*img_dim_inches*img_dpi
img_arr = np.zeros([xdim, ydim, 4])

X = np.zeros([xdim, ydim, 4])
result = np.ctypeslib.as_ctypes(np.zeros([xdim, ydim, 4]))
shared_array = sharedctypes.RawArray(result._type_, result)


def fill_per_window(event):
    colnum = int(max_cols - (np.ceil(event/max_cols)*max_cols - event))%max_cols
    rownum = np.floor(event/max_cols)
    tmp = np.ctypeslib.as_array(shared_array)
    
    tmp_img = np.array(Image.open('./IMAGES/' + str(event).zfill(5) + '.png').getdata()).reshape(int(img_dim_inches*img_dpi), int(img_dim_inches*img_dpi), 4)

    row_from = int(rownum*img_dim_inches*img_dpi)
    row_to   = int(rownum*img_dim_inches*img_dpi + img_dim_inches*img_dpi)
    col_from = int(colnum*img_dim_inches*img_dpi)
    col_to   = int(colnum*img_dim_inches*img_dpi + img_dim_inches*img_dpi)
    
    tmp[row_from:row_to, col_from:col_to, :] = tmp_img

In [429]:
p = Pool()
res = p.map(fill_per_window, range(num_examples))
result = np.ctypeslib.as_array(shared_array)
_ = p.close()

imgout = Image.fromarray(result.astype('uint8'))

In [430]:
imgout.save('atlas.png', 'PNG', transparent=False)

In [431]:
!cp atlas_test.png ./static/atlas_test.png

In [442]:
df_features['PredictedClass'] = None

In [443]:
jsonstr = df_features[metadata_columns].to_json(orient='records')

In [444]:
with open('./static/var_data_template.js', 'r') as fp:
    with open('./static/var_data.js', 'w') as fp2:
        var_data = fp.read()
        var_data = var_data.replace('{{data}}', jsonstr)
        fp2.write(var_data)